In [1]:
import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent.parent.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings
from chains import XRPQuerier, XRPProcessor, XRPPipeline
import logging
from database import MongoDatabase, MongoInsertOperations, MongoQueryOperations

logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')

from decoder import LogDecoder
from decoder2 import LogDecoder2
from config import Settings
from web3 import Web3



In [2]:
def decode_hex(value):
    """
    Decode a hexadecimal string to an integer if it's an Ethereum-style integer (e.g., block numbers, gas values).
    Does not decode long hashes or other non-integer hex values.
    :param value: Hexadecimal string (e.g., '0x677df92f') or other types.
    :return: Decoded integer or original value if not a valid short hex integer.
    """
    if isinstance(value, str) and value.startswith("0x"):
        if value == '0x':
            return None
        
        # Only decode if the hex string is short (e.g., block numbers, gas, timestamps)
        return int(value, 16)
    return value  # Return original value if not a short hex integer

# Parallelization of the tasks
def reconcile_logs_with_transactions(block):
    
    transactions = block["transactions"]
    w3 = Web3(Web3.HTTPProvider(Settings.ETHEREUM_ENDPOINT))
    
    # Fetch logs for the block
    logs = w3.eth.get_logs({
        "fromBlock": decode_hex(block['number']),
        "toBlock": decode_hex(block['number'])
    })

    # Group logs by transaction hash
    logs_by_transaction = {}
    for log in logs:
        tx_hash = log["transactionHash"].to_0x_hex()
        if tx_hash not in logs_by_transaction:
            logs_by_transaction[tx_hash] = []
        logs_by_transaction[tx_hash].append(dict(log))

    # Analyze the results
    tx_with_logs = len(logs_by_transaction)
    tx_without_logs = len(transactions) - tx_with_logs

    
    
    print(f"Total transactions in block: {len(transactions)}")
    print(f"Transactions with logs: {tx_with_logs}")
    print(f"Transactions without logs: {tx_without_logs}")

    # Return grouped logs for further analysis
    return logs_by_transaction



In [3]:
mongodb = MongoDatabase()
mongodb_query_ops = MongoQueryOperations(mongodb)

queries = mongodb_query_ops.get_recent_blocks('Ethereum', 100)

query = mongodb_query_ops.get_block_by_number('Ethereum', 21585277)
data = query['raw_block_data']


2025-01-11 23:34:23,810 - INFO - Connected to MongoDB database: blockchain
2025-01-11 23:34:23,819 - INFO - Indexes created for collection: Solana
2025-01-11 23:34:23,822 - INFO - Indexes created for collection: Bitcoin
2025-01-11 23:34:23,823 - INFO - Indexes created for collection: Ethereum
2025-01-11 23:34:23,824 - INFO - Indexes created for collection: BNB
2025-01-11 23:34:23,825 - INFO - Indexes created for collection: XRP
2025-01-11 23:34:23,837 - INFO - Retrieved 100 most recent blocks from the Ethereum collection.
2025-01-11 23:34:23,973 - INFO - Retrieved block 21585277 from Ethereum collection in MongoDB.


In [4]:
logs_by_transaction = reconcile_logs_with_transactions(queries[90]['raw_block_data'])

Total transactions in block: 215
Transactions with logs: 142
Transactions without logs: 73


In [5]:
logs_by_transaction

{'0x5aebdd1e8ced854695a17905ca0a1fb92d5f61aabd80c8d2905bbc66fd3b6f95': [{'address': '0xcA4F53E6117623992126A9A45Ce61682FE8678Df',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x0000000000000000000000001f2f10d1c40777ae1da742455c65828ff36df387'),
    HexBytes('0x0000000000000000000000006384b1e35ee28662e7460ceab10eb3d6a265792b')],
   'data': HexBytes('0x0000000000000000000000000000000000000000000000000000090e7a6d0000'),
   'blockNumber': 21602701,
   'transactionHash': HexBytes('0x5aebdd1e8ced854695a17905ca0a1fb92d5f61aabd80c8d2905bbc66fd3b6f95'),
   'transactionIndex': 0,
   'blockHash': HexBytes('0x18ad3c49b017644cac923e2ff4543135c4293e2d38ce812e8eaa9d5ba95ad06f'),
   'logIndex': 0,
   'removed': False},
  {'address': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x0000000000000000000000006384b1e35ee28662e7460ceab10eb3

In [6]:
log_decoder2 = LogDecoder2(Settings.ETHERSCAN_API_KEY)

results = log_decoder2.process_transaction_logs(logs_by_transaction)

results


No event signature found for log: {'address': '0x70bf6634eE8Cb27D04478f184b9b8BB13E5f4710', 'topics': [], 'data': HexBytes('0x7bb8d89a2de56508c007e1f8b3de4b1f236c406a89e062e955726be6503e2bbe000000000000000000324ad7e6a8934d'), 'blockNumber': 21602701, 'transactionHash': HexBytes('0x8cff0852685a95d2dfb83d3408965d13ad5dc61322bb1bf4b883adff626cce78'), 'transactionIndex': 185, 'blockHash': HexBytes('0x18ad3c49b017644cac923e2ff4543135c4293e2d38ce812e8eaa9d5ba95ad06f'), 'logIndex': 515, 'removed': False}
No event signature found for log: {'address': '0x70bf6634eE8Cb27D04478f184b9b8BB13E5f4710', 'topics': [], 'data': HexBytes('0x7b6fe44011ae012494e3ab6f202a0469444594b9235cc9de0de568de83521d32000000000000000000000000042d1f57'), 'blockNumber': 21602701, 'transactionHash': HexBytes('0x024e7884eab64af730a16a1943782e1fb1dfacffcceb2e4be085c9a043a53974'), 'transactionIndex': 188, 'blockHash': HexBytes('0x18ad3c49b017644cac923e2ff4543135c4293e2d38ce812e8eaa9d5ba95ad06f'), 'logIndex': 535, 'removed': F

{'0x5aebdd1e8ced854695a17905ca0a1fb92d5f61aabd80c8d2905bbc66fd3b6f95': [{'from': '0x1f2f10d1c40777ae1da742455c65828ff36df387',
   'to': '0x6384b1e35ee28662e7460ceab10eb3d6a265792b',
   'value': 9957788155904,
   'event': 'Transfer',
   'contract': '0xcA4F53E6117623992126A9A45Ce61682FE8678Df'},
  {'src': '0x6384b1e35ee28662e7460ceab10eb3d6a265792b',
   'dst': '0x1f2f10d1c40777ae1da742455c65828ff36df387',
   'wad': 69805835949178880,
   'event': 'Transfer',
   'contract': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'},
  {'reserve0': 27414736143510065278,
   'reserve1': 3908932001348389,
   'event': 'Sync',
   'contract': '0x6384b1E35EE28662E7460cEab10EB3D6a265792b'},
  {'sender': '0x1f2f10d1c40777ae1da742455c65828ff36df387',
   'to': '0x1f2f10d1c40777ae1da742455c65828ff36df387',
   'amount0In': 0,
   'amount1In': 9957788155904,
   'amount0Out': 69805835949178880,
   'amount1Out': 0,
   'event': 'Swap',
   'contract': '0x6384b1E35EE28662E7460cEab10EB3D6a265792b'}],
 '0x08b5ea96422bc1e6f9c

In [7]:
len(results)


142

In [6]:
log_decoder = LogDecoder(Settings.ETHERSCAN_API_KEY)

results = log_decoder.process_transaction_logs(logs_by_transaction)

No event signature found for log: {'address': '0x70bf6634eE8Cb27D04478f184b9b8BB13E5f4710', 'topics': [], 'data': HexBytes('0x7bb8d89a2de56508c007e1f8b3de4b1f236c406a89e062e955726be6503e2bbe000000000000000000324ad7e6a8934d'), 'blockNumber': 21602701, 'transactionHash': HexBytes('0x8cff0852685a95d2dfb83d3408965d13ad5dc61322bb1bf4b883adff626cce78'), 'transactionIndex': 185, 'blockHash': HexBytes('0x18ad3c49b017644cac923e2ff4543135c4293e2d38ce812e8eaa9d5ba95ad06f'), 'logIndex': 515, 'removed': False}
No event signature found for log: {'address': '0x70bf6634eE8Cb27D04478f184b9b8BB13E5f4710', 'topics': [], 'data': HexBytes('0x7b6fe44011ae012494e3ab6f202a0469444594b9235cc9de0de568de83521d32000000000000000000000000042d1f57'), 'blockNumber': 21602701, 'transactionHash': HexBytes('0x024e7884eab64af730a16a1943782e1fb1dfacffcceb2e4be085c9a043a53974'), 'transactionIndex': 188, 'blockHash': HexBytes('0x18ad3c49b017644cac923e2ff4543135c4293e2d38ce812e8eaa9d5ba95ad06f'), 'logIndex': 535, 'removed': F

In [7]:
log_decoder = LogDecoder(Settings.ETHERSCAN_API_KEY)

results = log_decoder.process_transaction_logs(logs_by_transaction)

No event signature found for log: {'address': '0x70bf6634eE8Cb27D04478f184b9b8BB13E5f4710', 'topics': [], 'data': HexBytes('0x7bb8d89a2de56508c007e1f8b3de4b1f236c406a89e062e955726be6503e2bbe000000000000000000324ad7e6a8934d'), 'blockNumber': 21602701, 'transactionHash': HexBytes('0x8cff0852685a95d2dfb83d3408965d13ad5dc61322bb1bf4b883adff626cce78'), 'transactionIndex': 185, 'blockHash': HexBytes('0x18ad3c49b017644cac923e2ff4543135c4293e2d38ce812e8eaa9d5ba95ad06f'), 'logIndex': 515, 'removed': False}
No event signature found for log: {'address': '0x70bf6634eE8Cb27D04478f184b9b8BB13E5f4710', 'topics': [], 'data': HexBytes('0x7b6fe44011ae012494e3ab6f202a0469444594b9235cc9de0de568de83521d32000000000000000000000000042d1f57'), 'blockNumber': 21602701, 'transactionHash': HexBytes('0x024e7884eab64af730a16a1943782e1fb1dfacffcceb2e4be085c9a043a53974'), 'transactionIndex': 188, 'blockHash': HexBytes('0x18ad3c49b017644cac923e2ff4543135c4293e2d38ce812e8eaa9d5ba95ad06f'), 'logIndex': 535, 'removed': F

In [7]:
results = []
for i, item in enumerate(queries[10:20]):
    print(f"Processing block {i+10} of {len(queries)}")
    logs_by_transaction = reconcile_logs_with_transactions(item['raw_block_data'])
    results = log_decoder.process_transaction_logs(logs_by_transaction)
    results.append(results)

print(log_decoder.known_events)

Processing block 10 of 100
Total transactions in block: 158
Transactions with logs: 101
Transactions without logs: 57


IndexError: list index out of range

In [2]:
logs_by_transaction


NameError: name 'logs_by_transaction' is not defined

In [28]:
log_decoder.known_events

{'ddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef': 'Transfer(address,address,uint256)',
 'c42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67': 'Swap(address,address,int256,int256,uint160,uint128,int24)',
 'd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822': 'Swap(address,uint256,uint256,uint256,uint256,address)',
 '48e9e2c732ba278de6ac88a3a57a5c5ba13d3d8370e709b3b98333a57876ca95': 'NewReport(bytes32,uint256,bytes,uint256,bytes,address)',
 'e1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c': 'Deposit(address,uint256)',
 '8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925': 'Approval(address,address,uint256)',
 '5fe47ed6d4225326d3303476197d782ded5a4e9c14f479dc9ec4992af4e85d59': 'Deposit(address,address,address,uint256,uint256)',
 '1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1': 'Sync(uint112,uint112)',
 '7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98cb3bf7268a95bf5081b65': 'Withdrawal(address,uint